# Testing Agentic Planning with ControlFlow

In [1]:
from datetime import datetime
from typing import List, Optional

import controlflow as cf
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

True

In [2]:
reply = cf.run(
    "Chunk a project into smaller tasks",
    context=dict(project="Get a job as a genAI engineer"),
)

print(reply)

Output()

1. **Research and Study GenAI**:
    - Understand what generative AI (genAI) encompasses.
    - Identify key skills and concepts required for a GenAI engineer.

2. **Skill Acquisition**:
    - Learn programming languages commonly used in AI, such as Python.
    - Study machine learning and deep learning fundamentals.
    - Gain expertise in frameworks like TensorFlow and PyTorch.
    - Develop understanding in AI model development and deployment.

3. **Build a Portfolio**:
    - Work on GenAI projects or simulations, such as creating generative models or AI tools.
    - Document projects, detailing the problem-solving process and outcomes.
    - Publish projects on platforms like GitHub for visibility.

4. **Networking and Community Involvement**:
    - Join GenAI and AI-related meetups, forums, or online communities.
    - Attend webinars, workshops, and conferences.
    - Connect with current GenAI professionals on LinkedIn or other platforms.

5. **Apply for Jobs**:
    - Update res

In [3]:
# Create a specialized agent
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions="You are a project planning expert using the Getting Things Done \
        (GTD) methodology. When given a project or task, chunk it down into a \
        hierarchical tree of subtasks.",
)


# Set up a ControlFlow task to classify emails
tasks = cf.run(
    "Chunk the project into a hierarchical task tree",
    # result_type=TaskList,
    agents=[chunker],
    context=dict(project="Get a job as a genAI engineer"),
)

print(tasks)

Output()

1. Update resume
   1.1 Gather current job experiences
   1.2 Update skills and educational background
   1.3 Tailor resume for GenAI roles
   1.4 Format and proofread resume
2. Build portfolio
   2.1 Identify projects relevant to GenAI
   2.2 Create or refine case studies
   2.3 Upload projects to a personal website or GitHub
3. Networking
   3.1 Join GenAI communities online
   3.2 Attend industry conferences and meetups
   3.3 Reach out to professionals in the field
4. Job applications
   4.1 Search for job openings
   4.2 Apply to positions with tailored resumes
   4.3 Follow up on applications
5. Prepare for interviews
   5.1 Research common interview questions in GenAI
   5.2 Conduct mock interviews
   5.3 Develop a personal pitch for interviews


In [4]:
# Create a specialized agent
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions="You are a project planning expert using the Getting Things Done \
        (GTD) methodology. When given a project or task, chunk it down into a \
        hierarchical tree of subtasks.",
)


class ChunkedTask(BaseModel):
    id: str = Field(description="Unique identifier for the task")
    parent_id: Optional[str] = Field(
        None, description="ID of parent task, null if root"
    )
    title: str = Field(description="Title that captures the task's action/outcome")
    created_at: datetime = Field(default_factory=datetime.utcnow)


class TaskList(BaseModel):
    tasks: List["ChunkedTask"] = Field(default_factory=list)


# Set up a ControlFlow task to classify emails
chunk_task = cf.Task(
    objective="Chunk the project into a hierarchical task tree",
    result_type=TaskList,
    agents=[chunker],
    context=dict(project="Get a job as a genAI engineer"),
)

tasks = chunk_task.run()

print(tasks)

Output()

tasks=[ChunkedTask(id='1', parent_id=None, title='Get a job as a genAI engineer', created_at=datetime.datetime(2025, 2, 12, 16, 14, 47, 462087)), ChunkedTask(id='1.1', parent_id='1', title='Update resume', created_at=datetime.datetime(2025, 2, 12, 16, 14, 47, 462087)), ChunkedTask(id='1.1.1', parent_id='1.1', title='Review current resume', created_at=datetime.datetime(2025, 2, 12, 16, 14, 47, 462087)), ChunkedTask(id='1.1.2', parent_id='1.1', title='Add relevant projects and experiences', created_at=datetime.datetime(2025, 2, 12, 16, 14, 47, 462087)), ChunkedTask(id='1.1.3', parent_id='1.1', title='Tailor resume for specific job applications', created_at=datetime.datetime(2025, 2, 12, 16, 14, 47, 462087)), ChunkedTask(id='1.2', parent_id='1', title='Build portfolio', created_at=datetime.datetime(2025, 2, 12, 16, 14, 47, 462087)), ChunkedTask(id='1.2.1', parent_id='1.2', title='Select projects to showcase', created_at=datetime.datetime(2025, 2, 12, 16, 14, 47, 462087)), ChunkedTask(id='

## Multi-Agent Solution
The flow we're eying atm would be this:
```
A: input P -> B [find Ps without NA children]
B -> C{Any found?}
C -> |yes| D[generate children for these Ps]
D -> E[Classify new children as P or NA]
E -> D
C -> |no| F[Check if each P's direct children achieve its outcome]
F -> |Incomplete Ps| D
F -> |all complete| G[Final Task Tree]
```

But let's build towards this gradually.






In [5]:
def print_task_tree(tasks: TaskList, parent_id: str = "1", indent: int = 0):
    """Print tasks in a hierarchical tree format.

    Args:
        tasks: TaskList containing all tasks
        parent_id: ID of parent task to start from (defaults to "1")
        indent: Current indentation level
    """
    # Find tasks with this parent
    current_level = [t for t in tasks.tasks if t.parent_id == parent_id]

    # Print each task at this level
    for task in current_level:
        print("    " * indent + f"- {task.title}")
        print_task_tree(tasks, task.id, indent + 1)

### V0.1.1 - 1 agent and 1 task to chunk a project

In [6]:
class TaskPNA(BaseModel):
    id: str = Field(description="unique identifier for the task")
    title: str = Field(description="A short title/description of the task")
    parent_id: Optional[str] = Field(
        None, description="ID of this task's parent task, null if root"
    )


class TaskList(BaseModel):
    tasks: List[TaskPNA] = Field(default_factory=list)

In [7]:
# Create a specialized agent
chunker_instructions = """
You are a Getting Things Done project management expert, specialized into
subdividing big tasks into smaller ones.
"""
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions=chunker_instructions,
)


@cf.flow
def chunk_flow_v0_1_1(project: str):
    # Initialize with root project
    root_task_id = "1"
    tasks = TaskList(tasks=[TaskPNA(id=root_task_id, title=project, parent_id=None)])

    with cf.Task(
        "Generate hyrarchical task tree for this project",
        result_type=TaskList,
        agents=[chunker],
    ) as main_task:
        new_tasks = cf.run(
            """
            Break given task (with id {task_id}) into subtasks.
            Ensure that any subtask saves the task_id of its parent task as its
            parent_id.
            Once you split the task into subtasks, mark this task as complete and
            use the main task tool to mark that as complete.
            """,
            context=dict(
                task_id=root_task_id,
                tasks=tasks,
            ),
            agents=[chunker],
            result_type=TaskList,
            tools=[main_task.get_success_tool()],
        )

        tasks = TaskList(tasks=tasks.tasks + new_tasks.tasks)

    return tasks


tasks = chunk_flow_v0_1_1("Get a job as a genAI engineer")

print(tasks)

Output()

tasks=[TaskPNA(id='1', title='Get a job as a genAI engineer', parent_id=None), TaskPNA(id='1.1', title='Update resume to highlight GenAI skills', parent_id='1'), TaskPNA(id='1.2', title='Research job openings related to GenAI engineering', parent_id='1'), TaskPNA(id='1.3', title='Prepare for interviews by practicing common questions', parent_id='1'), TaskPNA(id='1.4', title='Network with professionals in the GenAI field', parent_id='1')]


In [8]:
print(tasks)

tasks=[TaskPNA(id='1', title='Get a job as a genAI engineer', parent_id=None), TaskPNA(id='1.1', title='Update resume to highlight GenAI skills', parent_id='1'), TaskPNA(id='1.2', title='Research job openings related to GenAI engineering', parent_id='1'), TaskPNA(id='1.3', title='Prepare for interviews by practicing common questions', parent_id='1'), TaskPNA(id='1.4', title='Network with professionals in the GenAI field', parent_id='1')]


In [9]:
print([f"{t.id}: {t.title} (parent: {t.parent_id})" for t in tasks.tasks])

['1: Get a job as a genAI engineer (parent: None)', '1.1: Update resume to highlight GenAI skills (parent: 1)', '1.2: Research job openings related to GenAI engineering (parent: 1)', '1.3: Prepare for interviews by practicing common questions (parent: 1)', '1.4: Network with professionals in the GenAI field (parent: 1)']


In [10]:
print_task_tree(tasks)

- Update resume to highlight GenAI skills
- Research job openings related to GenAI engineering
- Prepare for interviews by practicing common questions
- Network with professionals in the GenAI field


### V0.1.2 - converting task list to tree of nodes

In [11]:
class TaskNode(BaseModel):
    id: str = Field(description="Unique identifier for the task")
    title: str = Field(description="Title/description of the task")
    subtasks: List["TaskNode"] = Field(default_factory=list)


TaskNode.model_rebuild()


# Result wrapper for ControlFlow
class TaskResult(BaseModel):
    task: TaskNode

In [12]:
chunker_instructions = """
You are a Getting Things Done project management expert, specialized into
subdividing big tasks into smaller ones.
"""
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions=chunker_instructions,
)


@cf.flow
def chunk_flow_v0_1_2(project: str):
    # Initialize with root project
    root_task_id = "1"
    root_task = TaskNode(id=root_task_id, title=project)

    with cf.Task(
        "Generate hyrarchical task tree for this project",
        result_type=TaskResult,
        agents=[chunker],
    ) as main_task:
        new_tasks = cf.run(
            """
            Break given task into subtasks.
            Return a TaskResult containing a TaskNode with:
            - The root task
            - 3-5 subtasks in the subtasks list
            Once you split the task into subtasks, mark this task as complete and
            use the main task tool to mark that as complete.
            """,
            context=dict(
                task=root_task,
            ),
            agents=[chunker],
            result_type=TaskResult,
            tools=[main_task.get_success_tool()],
        )

        # tasks = TaskList(tasks=tasks.tasks + new_tasks.tasks)

    return new_tasks.task


tasks = chunk_flow_v0_1_2("Get a job as a genAI engineer")

print(tasks)

Output()

id='1' title='Get a job as a genAI engineer' subtasks=[TaskNode(id='1.1', title='Update resume and portfolio', subtasks=[]), TaskNode(id='1.2', title='Research job openings in AI engineering', subtasks=[]), TaskNode(id='1.3', title='Prepare for common interview questions', subtasks=[]), TaskNode(id='1.4', title='Network with professionals in the industry', subtasks=[]), TaskNode(id='1.5', title='Apply to selected job openings', subtasks=[])]


In [19]:
def print_task_nodes(task: TaskNode, indent: int = 0):
    """Print task tree recursively with nice formatting.

    Args:
        task: Task to print
        indent: Current indentation level
    """
    # Print current task with indentation
    print("    " * indent + f"{task.id} {task.title}")

    # Recursively print subtasks
    for subtask in task.subtasks:
        print_task_nodes(subtask, indent + 1)

In [20]:
print_task_nodes(tasks)

1 Get a job as a genAI engineer
    1.1 Update resume and portfolio
    1.2 Research job openings in AI engineering
    1.3 Prepare for common interview questions
    1.4 Network with professionals in the industry
    1.5 Apply to selected job openings
